In [ ]:
import numpy as np
from numba import njit
import matplotlib.pyplot as plt
plt.rcParams.update({'pdf.fonttype': 42, 'ps.fonttype': 42})
plt.rcParams.update({'font.size': 10, 'font.family': 'Helvetica'})
from math import gamma

cutsQ = True
MeanAnglesQ = True
MedianAnglesQ = True
AverageRadiusNormalisedQ = True
MaxRadiusNormalisedQ = True
MedianRadiusNormalisedQ = True
AverageRadiusQ = True
MaxRadiusQ = True
MedianRadiusQ = True
GaborQ = True
s_mag = 3

ticks = 'Log' # 'True', 'Log'
x_axis = 'Redundancy' # 'Redundancy' 'NumberNeurons'
x_axis_label = 'Redundancy' # 'Redundancy' 'NumberNeurons'

color_cube = 'blue'
color_box = 'darkorange'
color_sphere = 'green'

In [ ]:
polar_points = 1000

no_dim, min_dim, max_dim = 17, 2, 100
no_red, min_red, max_red = 17, 2, 100
no_non, min_non, max_non = 17, 2, 1000

starting_dist_angles = 1000
min_dist_angles = 100

starting_dist_volume = 1000000
min_dist_volume = 10000

manual_runs = [] 

save = True

In [ ]:
@njit
def random_network(dim, N, seed=1):
    '''Creates a Random Network, dim is Signal Dimensionality, N is the number of neurons'''
    np.random.seed(seed)
    D = np.random.randn(dim, N)
    for i in range(N):
        D[:, i] /= np.linalg.norm(D[:, i])
    Th = 0.5 * np.ones(N)
    return D, Th
def orth(u):
    '''given input vector u, find a random orthogonal vector w'''
    v = np.random.randn(u.shape[0])
    w = v - u * np.dot(u, v) / np.dot(u, u)
    return w / np.linalg.norm(w)
def polar(A, b, u, v, no_ang):
    angles = np.linspace(0, 2 * np.pi, no_ang)
    r = np.zeros(no_ang)
    dim = A.shape[1]
    for i, ang in enumerate(angles):
        ray = np.cos(ang) * u + np.sin(ang) * v
        Aray = np.dot(A, ray)
        a = [b[i] / Aray[i] if Aray[i] > 0 else np.inf for i in range(b.shape[0])]
        r[i] = np.min(a)
        r[i] = min(r[i], s_mag * np.sqrt(2) * gamma((dim + 1) / 2) / gamma(dim / 2))
    return r, angles
def dot_adjacent(A, b, accuracy):
    dim = A.shape[1]
    if dim == 2:
        j, k = np.argsort(np.arctan2(A[:, 0], A[:, 1]))[0:2]
    else:
        k = rnd_boundary_point(A, b)
        j = find_neighbour(A, b, k, accuracy)
    return np.dot(A[j, :], A[k, :])
def rnd_boundary_point(A, b):
    dim = A.shape[1]
    N = b.shape[0]
    not_yet = 1
    while not_yet:
        ray = np.random.randn(dim)
        Aray = np.dot(A, ray)
        a = [b[i] / Aray[i] if Aray[i] > 0 else np.inf for i in range(N)]
        if np.min(a) != np.inf:
            not_yet = 0
    return np.argmin(a)
def find_neighbour(A, b, k, accuracy):
    low_theta = 0
    high_theta = np.pi / 2
    dim = A.shape[1]
    j = k
    ite = 0
    sucess = 0
    while sucess == 0 and ite < 100:
        orth_ray = orth(A[k, :])
        j, sucess = boundary_point(A, b, orth_ray)
        ite += 1
    if ite == 100:
        print('Not a single of the 100 random rotations got you to a different neuron. The following number should be -1: {}'.format(j))
    if j == k:
        print(A[k, :], orth_ray, np.dot(A[k, :], orth_ray), ite)
    while(high_theta - low_theta > accuracy):
        test_theta = (low_theta + high_theta) / 2
        ray = np.cos(test_theta) * A[k, :] + np.sin(test_theta) * orth_ray
        j, sucess = boundary_point(A, b, ray)
        if j != k:
            high_theta = test_theta
        else:
            low_theta = test_theta
    j, sucess = boundary_point(A, b, np.cos(high_theta) * A[k, :] + np.sin(high_theta) * orth_ray)
    if high_theta > np.pi / 2 - accuracy:
        print('hole')
    if j == k:
        print('? again?')
    return j
def boundary_point(A, b, ray, max_sight = s_mag):
    '''
        finds whether or not the system Ax <= b when x = k * ray will eventually be violated and by what bound
    '''
    N = b.shape[0]
    dim = A.shape[1]
    not_yet = 1
    Aray = np.dot(A, ray)
    a = [b[i] / Aray[i] if Aray[i] > 0 else max_sight * np.sqrt(2) * gamma((dim + 1) / 2) / gamma(dim / 2) for i in range(N)]
    if np.min(a) == max_sight * np.sqrt(2) * gamma((dim + 1) / 2) / gamma(dim / 2):
        return -1, 0
    else:
        return np.argmin(a), 1

# RADIUS SECTION
@njit
def cube_radius(dim, seed, no_rays):
    np.random.seed(seed)
    D_c = np.hstack((np.eye(dim), -np.eye(dim)))
    Th_c = 0.5 * np.ones(2 * dim)
    radius_c = np.zeros(no_rays)
    for i in range(no_rays):
        ray = np.random.randn(dim)
        ray /= np.linalg.norm(ray)
        Aray = np.dot(-D_c.T, ray)
        a = [Th_c[j] / Aray[j] if Aray[j] > 0 else np.inf for j in range(2 * dim)]
        radius_c[i] = min(np.min(np.array([a])), s_mag * np.sqrt(2) * gamma((dim + 1) / 2) / gamma(dim / 2)) # this is the expected value for the chi distribution
    mean_radius_c = np.mean(radius_c) # Here we could also say something analytically. But the same argument applies
    max_radius_c = np.max(radius_c) # here we could say that this is np.sqrt(dim). But it's better if we let our random sampling do the job
    median_radius_c = np.median(radius_c) # Here we could also say something analytically. But the same argument applies
    return mean_radius_c, max_radius_c, median_radius_c

@njit
def box_radius(dim, N, seed, no_rays):
    np.random.seed(seed)
    radius = np.zeros(no_rays)
    for i in range(no_rays):
        D, Th = random_network(dim, N, seed=i)
        ray = np.random.randn(dim)
        ray /= np.linalg.norm(ray)
        Aray = np.dot(-D.T, ray)
        a = [Th[j] / Aray[j] if Aray[j] > 0 else np.inf for j in range(N)]
        radius[i] = min(np.min(np.array([a])), s_mag * np.sqrt(2) * gamma((dim + 1) / 2) / gamma(dim / 2))
    mean_radius = np.mean(radius)
    max_radius = np.max(radius)
    median_radius = np.median(radius)
    return mean_radius, max_radius, median_radius

# 2D cuts of EBB with hyper-cube and hypersphere

In [ ]:
if cutsQ:
    np.random.seed(2)
    DIM = [2, 10, 20, 100]
    if x_axis == 'Redundancy':
        AUX = [2, 10, 20, 100]
    else:
        AUX = [2, 20, 200, 2000]
    nocuts = [1, 1, 1, 1]
    fig, ax = plt.subplots(len(DIM), len(AUX), figsize=(10, 10))
    for i, dim in enumerate(DIM[::-1]):
        for j, aux in enumerate(AUX):
            this_ax = ax[i, j]
            if x_axis == 'Redundancy':
                D, Th = random_network(dim, dim * aux)
            else:
                D, Th = random_network(dim, aux)
            D_cube = np.hstack((np.eye(dim), -np.eye(dim)))
            Th_cube = 0.5 * np.ones(2 * dim)
            for k in range(nocuts[i]):
                u = np.random.randn(dim)
                u /= np.linalg.norm(u)
                v = orth(u)
                v /= np.linalg.norm(v)
                r, angles = polar(-D.T, Th, u, v, polar_points)
                r_cube, angles_cube = polar(-D_cube.T, Th_cube, u, v, polar_points)
                this_ax.plot(np.cos(angles) / 2, np.sin(angles) / 2, ':', color=color_sphere) # plotting the hypersphere
                this_ax.plot(r * np.cos(angles), r * np.sin(angles), alpha = 0.9 ** nocuts[i], color=color_box) # plotting the bounding box
                this_ax.plot(r_cube * np.cos(angles_cube), r_cube * np.sin(angles_cube), ':', alpha = 0.9 ** nocuts[i], color=color_cube)  # plotting the hypercube
                this_ax.set_xlim([-2.2, 2.2])
                this_ax.set_ylim([-2.2, 2.2])
                if dim == DIM[0] and j == 0:
                    this_ax.set_xticks([-1, 1])
                    this_ax.set_yticks([-1, 1])
                    this_ax.spines['right'].set_visible(False)
                    this_ax.spines['top'].set_visible(False)
                else:
                    this_ax.set_aspect('equal')
                    this_ax.set_axis_off()
    # plt.tight_layout()
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.savefig('SFIG3.pdf', transparent=True)

Max L2 dist inside EBB compared with n-cube and n-sphere

Mean angle between neighbouring neurons

In [ ]:
dict_result = {}
DIM = np.unique(np.round(np.logspace(np.log10(min_dim), np.log10(max_dim), no_dim)).astype(int).tolist())
if x_axis == 'Redundancy':
    AUX = np.unique(np.round(np.logspace(np.log10(min_red), np.log10(max_red), no_red)).astype(int).tolist())
else:
    AUX = np.unique(np.round(np.logspace(np.log10(min_non), np.log10(max_non), no_non)).astype(int).tolist())
no_dim = len(DIM)
no_aux = len(AUX)
print(DIM, AUX)
metrics = []
LEVELS = []
VV = []
CMAPQ = []

In [ ]:
if MeanAnglesQ or MedianAnglesQ:
    if MeanAnglesQ:
        metrics = metrics + ['MeanAngles']
        LEVELS = LEVELS + [[0, np.pi / 6, np.pi / 4, 2 * np.pi / 6, 7 * np.pi / 18, 8 * np.pi / 18, np.pi / 2]]
        VV = VV + [[0, np.pi / 2]]
        CMAPQ = CMAPQ + ['Oranges'] # ['winter_r']
    if MedianAnglesQ:
        metrics = metrics + ['MedianAngles']
        LEVELS = LEVELS + [[0, np.pi / 6, np.pi / 4, 2 * np.pi / 6, 7 * np.pi / 18, 8 * np.pi / 18, np.pi / 2]]
        VV = VV + [[0, np.pi / 2]]
        CMAPQ = CMAPQ + ['Oranges'] # ['winter_r']
    accuracy = 0.0001

    if x_axis == 'Redundancy':
        decay_aux = 0.4
        max_aux = max_red
    else:
        decay_aux = 1
        max_aux = max_non
    decay_dim = 0.6
    size_dist = (min_dist_angles * np.ones((no_dim, no_aux))).astype(int)
    for i, dim in enumerate(DIM):
        for j, aux in enumerate(AUX):
            size_dist[i, j] = int(max(min_dist_angles, np.exp(-decay_dim * dim / max_dim) * np.exp(-decay_aux * aux / max_aux) * starting_dist_angles))
    print(size_dist)

    meanNA = np.zeros((no_dim, no_aux))
    medianNA = np.zeros((no_dim, no_aux))
    for i, dim in enumerate(DIM):
        for j, aux in enumerate(AUX):
            file_name = str(['aux/geometry/angle_between_neurons_box', x_axis, size_dist[i, j], dim, aux, accuracy]).replace('[', '').replace(']','').replace(', ','_').replace('.', 'dot').replace('\'', '')
            print(file_name, dim, aux)
            try:
                results = np.load(file_name + '.npz')
                dist_neighbors = results['dist_neighbors']
                print('sucess loading ', file_name)
            except:
                dist_neighbors = np.zeros(size_dist[i, j])
                for k in range(size_dist[i, j]):
                    if x_axis == 'Redundancy':
                        D, Th = random_network(dim, aux * dim, seed=k)
                    else:
                        D, Th = random_network(dim, aux, seed=k)
                    dist_neighbors[k] = dot_adjacent(-D.T, Th, accuracy)
                np.savez(file_name, dist_neighbors=dist_neighbors)
            meanNA[i, j] = np.median(np.arccos(np.clip(dist_neighbors, -1, 1)))
            medianNA[i, j] = np.median(np.arccos(np.clip(dist_neighbors, -1, 1)))
    dict_result['MeanAngles'] = meanNA
    dict_result['MedianAngles'] = medianNA

In [ ]:
if AverageRadiusNormalisedQ or MaxRadiusNormalisedQ or MedianRadiusNormalisedQ or AverageRadiusQ or MaxRadiusQ or MedianRadiusQ:
    if AverageRadiusNormalisedQ:
        metrics = metrics + ['AverageRadiusNormalised']
        LEVELS = LEVELS + [[-0.5, 0, 0.25, 0.5, 0.75, 1]]
        VV = VV + [[-1., 1.]]
        colors = [(0.1718, 0.637795276, 0.37109375), (0.16796875, 0.546875, 0.7421875), (0.88671875, 0.2890625, 0.19921875)]  # G -> B -> R 227,74,51
        CMAPQ = CMAPQ + ['Oranges'] # ['summer']
    if MaxRadiusNormalisedQ:
        metrics = metrics + ['MaxRadiusNormalised']
        LEVELS = LEVELS + [[-0.5, 0, 0.25, 0.5, 0.75, 1]]
        VV = VV + [[-1., 1.]]
        CMAPQ = CMAPQ + ['Oranges'] # ['summer']
    if MedianRadiusNormalisedQ:
        metrics = metrics + ['MedianRadiusNormalised']
        LEVELS = LEVELS + [[-0.5, 0, 0.25, 0.5, 0.75, 1]]
        VV = VV + [[-1., 1.]]
        CMAPQ = CMAPQ + ['Oranges'] # ['summer']
    if AverageRadiusQ:
        metrics = metrics + ['AverageRadius']
        LEVELS = LEVELS + [[0.5, 0.75, 1, 1.25, 1.5, 2]]
        VV = VV + [[0.5, 2]]
        LEVELS_HALF = [0.5]
        CMAPQ = CMAPQ + ['Oranges'] # ['summer']
    if MaxRadiusQ:
        metrics = metrics + ['MaxRadius']
        LEVELS = LEVELS + [[0.5, 0.75, 1, 1.25, 2, 3, 6]]
        VV = VV + [[0.5, 3]]
        LEVELS_HALF = [0.5]
        CMAPQ = CMAPQ + ['Oranges'] # ['summer']
    if MedianRadiusQ:
        metrics = metrics + ['MedianRadius']
        LEVELS = LEVELS + [[0.5, 0.6, 0.75, 1, 1.25, 1.5, 1.75]]
        VV = VV + [[0.5, 1.75]]
        LEVELS_HALF = [0.5]
        CMAPQ = CMAPQ + ['Oranges'] # ['summer']
    decay_dim = 2
    if x_axis == 'Redundancy':
        decay_aux = 0.4
        max_aux = max_red
    else:
        decay_aux = 1
        max_aux = max_non
    size_dist = (min_dist_volume * np.ones((no_dim, no_aux))).astype(int)
    for i, dim in enumerate(DIM):
        for j, aux in enumerate(AUX):
            size_dist[i, j] = int(max(min_dist_volume, np.exp(-decay_dim * dim / max_dim) * np.exp(-decay_aux * aux / max_aux) * starting_dist_volume))
    print(size_dist)
    if run == 'belem':
        for item in manual_runs:
            i_dim = np.where(DIM==item[0])[0]
            i_aux = np.where(AUX==item[1])[0]
            size_dist[i_dim, i_aux] = item[2]
    print(size_dist)
    
    ARCube = np.zeros((no_dim, no_aux))
    MRCube = np.zeros((no_dim, no_aux))
    MedRCube = np.zeros((no_dim, no_aux))
    AverageRadiusNormalised = np.zeros((no_dim, no_aux))
    MaxRadiusNormalised = np.zeros((no_dim, no_aux))
    MedianRadiusNormalised = np.zeros((no_dim, no_aux))
    AverageRadius = np.zeros((no_dim, no_aux))
    MaxRadius = np.zeros((no_dim, no_aux))
    MedianRadius = np.zeros((no_dim, no_aux))
    for i, dim in enumerate(DIM):
        total_seeds = size_dist[i, 0] * 10
        file_name = str(['aux/geometry/volume_cubes', total_seeds, dim]).replace('[', '').replace(']','').replace(', ','_').replace('.', 'dot').replace('\'', '')
        print(file_name, dim, aux)
        try:
            results = np.load(file_name + '.npz')
            arc = results['arc']
            mrc = results['mrc']
            medrc = results['medrc']
            print('sucess loading ', file_name)
        except:
            arc, mrc, medrc = cube_radius(dim, dim, total_seeds)
            np.savez(file_name, arc=arc, mrc=mrc, medrc=medrc)
        print('I have computed for the Cube in {} dim.'.format(dim))
        print('Its known maximum radius is {}, and we computed it to be {}.'.format(np.sqrt(dim) / 2, mrc))
        print('Its supposedly known average radius is about {}, and we computed it to be {}.'.format(np.sqrt(dim / (2 * np.pi * np.exp(1))), arc))
        print('We computed its median radius to be {}.'.format(medrc))
        for j, aux in enumerate(AUX):
            file_name = str(['aux/geometry/volume_box', x_axis, size_dist[i, j], dim, aux]).replace('[', '').replace(']','').replace(', ','_').replace('.', 'dot').replace('\'', '')
            print(file_name, dim, aux)
            try:
                results = np.load(file_name + '.npz')
                arn = results['arn']
                mrn = results['mrn']
                medrn = results['medrn']
                ar = results['ar']
                mr = results['mr']
                medr = results['medr']
                print('sucess loading ', file_name)
            except:
                if x_axis == 'Redundancy':
                    ar, mr, medr = box_radius(dim, aux * dim, (dim+1)*(aux+1), size_dist[i, j])
                else:
                    ar, mr, medr = box_radius(dim, aux, (dim+1)*(aux+1), size_dist[i, j])
                arn = (arc - ar) / (arc - 0.5)
                mrn = (mrc - mr) / (mrc - 0.5)
                medrn = (medrc - medr) / (medrc - 0.5)
                np.savez(file_name, arn=arn, mrn=mrn, medrn=medrn, ar=ar, mr=mr, medr=medr, arc=arc, mrc=mrc, medrc=medrc)
            AverageRadiusNormalised[i, j] = arn
            MaxRadiusNormalised[i, j] = mrn
            MedianRadiusNormalised[i, j] = medrn
            AverageRadius[i, j] = ar
            MaxRadius[i, j] = mr
            MedianRadius[i, j] = medr
            print('Average Radius {}, average radius of cube {}, normalisation {}'.format(ar, arc, arn))
            print('Max Radius {}, max radius of cube {}, normalisation {}'.format(mr, mrc, mrn))
            print('Median Radius {}, median radius of cube {}, normalisation {}'.format(medr, medrc, medrn))
        ARCube[i, AverageRadius[i, :] > arc] = 1
        MRCube[i, MaxRadius[i, :] > mrc] = 1
        MedRCube[i, MedianRadius[i, :] > medrc] = 1
    dict_result['AverageRadiusNormalised'] = AverageRadiusNormalised
    dict_result['MaxRadiusNormalised'] = MaxRadiusNormalised
    dict_result['MedianRadiusNormalised'] = MedianRadiusNormalised
    dict_result['AverageRadius'] = AverageRadius
    dict_result['MaxRadius'] = MaxRadius
    dict_result['MedianRadius'] = MedianRadius

    dict_result['ARCube'] = ARCube
    dict_result['MRCube'] = MRCube
    dict_result['MedRCube'] = MedRCube

In [ ]:
if MeanAnglesQ or MedianAnglesQ or AverageRadiusNormalisedQ or MaxRadiusNormalisedQ or MedianRadiusNormalisedQ or AverageRadiusQ or MaxRadiusQ or MedianRadiusQ:    
    
    alpha_b = 1
    
    if ticks == 'Log':
        base = 2
        DIM_x = np.log(DIM) / np.log(base)
        AUX_x = np.log(AUX) / np.log(base)
    else:
        DIM_x = DIM
        AUX_x = AUX
    
    if x_axis == 'Redundancy':
        max_aux = max_red
        min_aux = min_red
    else:
        max_aux = max_non
        min_aux = min_non
    no_cols = 6
    scaling_factor = 3
    fig, ax = plt.subplots(len(metrics), no_cols, figsize=(scaling_factor * no_cols, scaling_factor * len(metrics)))
    for i_metric, metric in enumerate(metrics):
        matrix = dict_result[metric]
        cmapQ = CMAPQ[i_metric]
        if VV[i_metric] == 'data':
            vmin, vmax = np.min(matrix), np.max(matrix)
        else:
            vmin, vmax = VV[i_metric][0], VV[i_metric][1]
        levels = LEVELS[i_metric]
        
        if metric == 'AverageRadius':
            matrix_contour = dict_result['ARCube']
        elif metric == 'MaxRadius':
            matrix_contour = dict_result['MRCube']
        elif metric == 'MedianRadius':
            matrix_contour = dict_result['MedRCube']
        
        # RAW data, line plots
        ax_plots = ax[i_metric, 0]
        for i_dim, dim in enumerate(DIM):
            ax_plots.plot(AUX_x, matrix[i_dim, :], label='dim:{}'.format(dim))
#         ax_plots.legend()
        ax_plots.set_xlabel(x_axis_label)
        ax_plots.set_xticklabels(AUX)
        if metric[-5:] == 'Angle':
            ax_plots.set_ylim([0, np.pi])
        ax_plots.set_title(metric)
        
        # RAW data, nuimshow
        ax_raw_data = ax[i_metric, 1]
        ax_raw_data.pcolormesh(AUX_x, DIM_x, matrix, shading='gouraud', alpha = alpha_b, cmap=cmapQ)
        ax_raw_data.set_xticks(AUX_x)
        ax_raw_data.set_yticks(DIM_x)
        ax_raw_data.set_xticklabels(AUX)
        ax_raw_data.set_yticklabels(DIM)
        
        # RAW data, nuimshow clipped
        ax_clip = ax[i_metric, 2]
        ax_clip.pcolormesh(AUX_x, DIM_x, matrix, shading='gouraud', vmin=vmin, vmax = vmax, alpha = alpha_b, cmap=cmapQ)
        ax_clip.set_xticks(AUX_x)
        ax_clip.set_yticks(DIM_x)
        ax_clip.set_xticklabels(AUX)
        ax_clip.set_yticklabels(DIM)
        ax_clip.set_aspect('equal')
        
#         matrix = np.clip(matrix, vmin, vmax)

        # Final, nuimshow clipped, contours, cube
        ax_final = ax[i_metric, 3]
        ax_final.pcolormesh(AUX_x, DIM_x, matrix, shading='gouraud', vmin=vmin, vmax = vmax, alpha = alpha_b, cmap=cmapQ)
        CS = ax_final.contour(AUX_x, DIM_x, matrix, levels=levels, linestyles=('-',), linewidths=(2,), colors='k')
        if metric == 'MeanAngles' or metric == 'MedianAngles':
            fmt = {}
            strs = ['0$^\circ$', '30$^\circ$', '45$^\circ$', '60$^\circ$', '70$^\circ$', '80$^\circ$', '90$^\circ$']
            for l,s in zip(CS.levels, strs):
                fmt[l] = s
            # Label every other level using strings
            ax_final.clabel(CS,CS.levels,inline=True,fmt=fmt)
        else:
            ax_final.clabel(CS)
        if metric == 'AverageRadius' or metric == 'MaxRadius' or metric == 'MedianRadius':
            CS_C = ax_final.contour(AUX_x, DIM_x, matrix_contour, levels=LEVELS_HALF, linestyles=('-',), linewidths=(2,), vmin=0, vmax = 1, colors=color_cube)
        if ticks == 'Log':
            ax_final.set_xticks(np.log(np.array([min_aux, 10, max_aux])) / np.log(base))
            ax_final.set_yticks(np.log(np.array([min_dim, 10, max_dim])) / np.log(base))
        else:
            ax_final.set_xticks([min_aux, 10, max_aux])
            ax_final.set_yticks([min_dim, 10, max_dim])
        ax_final.set_xticklabels([min_aux, 10, max_aux])
        ax_final.set_yticklabels([min_dim, 10, max_dim])
        ax_final.set_xlabel(x_axis_label)
        ax_final.set_ylabel('Dimensionality')
        

        ax_cb = ax[i_metric, -2]
        CSfake = ax_cb.imshow(vmin + (vmax - vmin) * (np.random.rand(matrix.shape[0], matrix.shape[1])<0.5), interpolation='nearest', cmap=cmapQ, alpha = alpha_b)
        ax_cbs = ax[i_metric, -1]
        cbar = fig.colorbar(CSfake, cax=ax_cbs, ticks=levels)
        cbar.add_lines(CS)
        ax_cbs.set_aspect(16)
        if metric == 'MeanAngles' or metric == 'MedianAngles':
            cbar.ax.set_yticklabels(['0$^\circ$', '30$^\circ$', '45$^\circ$', '60$^\circ$', '70$^\circ$', '80$^\circ$', '90$^\circ$'])
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    if save:
        out_file = 'SFIG3-1.pdf'.format(x_axis, ticks)
        fig.savefig(out_file, transparent=True)

# Gabor Patches

In [ ]:
if GaborQ:
    def gabor_fn(sigma, theta, Lambda, psi, gamma, no_squares, center):
        sigma_x = sigma
        sigma_y = sigma / gamma

        dxy = int(no_squares / 4)
        if center == 0:
            unitx = dxy
            unity = dxy
        elif center == 1:
            unitx = 2 * dxy
            unity = dxy
        elif center == 2:
            unitx = 3 * dxy
            unity = dxy
        elif center == 3:
            unitx = dxy
            unity = 2 * dxy
        elif center == 4:
            unitx = 2 * dxy
            unity = 2 * dxy
        elif center == 5:
            unitx = 3 * dxy
            unity = 2 * dxy
        elif center == 6:
            unitx = dxy
            unity = 3 * dxy
        elif center == 7:
            unitx = 2 * dxy
            unity = 3 * dxy
        elif center == 8:
            unitx = 3 * dxy
            unity = 3 * dxy

        (y, x) = np.meshgrid(np.arange(-unity, -unity + no_squares + 1), np.arange(-unitx, -unitx + no_squares + 1))
        # Rotation 
        x_theta = x * np.cos(theta) + y * np.sin(theta)
        y_theta = -x * np.sin(theta) + y * np.cos(theta)

        gb = np.exp(-.5 * (x_theta ** 2 / sigma_x ** 2 + y_theta ** 2 / sigma_y ** 2)) * np.cos(2 * np.pi * x_theta / Lambda + psi)
        return gb
    def angle(x, y):
        return np.arccos(np.clip(np.dot(x / np.linalg.norm(x), y / np.linalg.norm(y)), -1.0, 1.0))

In [ ]:
if GaborQ:
    no_pixels = 12
    gb = gabor_fn(1, np.pi / 4, 10, np.pi / 2, 1, no_pixels, 0)
    fig, ax = plt.subplots(1, 1, figsize=(3, 3), gridspec_kw = {'wspace':0, 'hspace':0})
    amp = np.max(np.abs(gb))
    ax.imshow(gb, cmap='binary', vmin=-amp, vmax=amp)
    ax.set_aspect('auto')
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
if GaborQ:
    no_rows = 10
    no_cols = 10
    np.random.seed(1)

    widths = [0.75, 1, 1.5]
    freq = [1, 2, 5, 10]
    Gamma = [0.75, 1, 1.5]
    Psi = [np.pi / 2]
    
    widths = [1, 1.5]
    freq = [3, 5, 10]
    Gamma = [1, 1.5]
    Psi = [np.pi / 2]

    GBP = [0 for i in range(no_cols * no_rows)]
    angles = np.zeros(no_cols * no_rows)
    fig, ax = plt.subplots(no_rows, no_cols, figsize=(3*no_cols, 3*no_rows), gridspec_kw = {'wspace':0, 'hspace':0})
    for i in range(no_rows):
        for j in range(no_cols):
            GBP[i * no_cols + j] = gabor_fn(np.random.choice(widths), np.random.rand() * 2 * np.pi, np.random.choice(freq), np.random.choice(Psi), np.random.choice(Gamma), no_pixels, np.random.randint(9))
#             GBP[i * no_cols + j] = gabor_fn(np.random.choice(widths), np.random.rand() * 2 * np.pi, np.random.choice(freq), np.random.rand() * 2 * np.pi, np.random.choice(Gamma), no_pixels, np.random.randint(9))
            amp = np.max(np.abs(GBP[i * no_cols + j]))
            ax[i, j].imshow(GBP[i * no_cols + j], cmap='binary', vmin=-amp, vmax=amp)
#             ax[i, j].imshow(GBP[i * no_cols + j], cmap='binary')
            ax[i, j].set_aspect('auto')
            ax[i, j].set_xticks([])
            ax[i, j].set_yticks([])
            angles[i * no_cols + j] = angle(gb.reshape(-1), GBP[i * no_cols + j].reshape(-1))

In [ ]:
if GaborQ:
    np.random.seed(1)

    quasi_ortho_deg = 5
    quasi_ortho_rad = quasi_ortho_deg * np.pi / 180
    angle_diff = np.abs(angles - np.pi / 2)
    gp_aligned = np.where(angle_diff > quasi_ortho_rad)[0]
    gp_aligned = np.random.choice(gp_aligned, 3, replace=False)
    gp_quasi = np.where(angle_diff <= quasi_ortho_rad)[0]
    gp_quasi = np.random.choice(gp_quasi, 18, replace=False)
    
    
    scale = 0.358083168

    no_rows_aligned = 1
    no_rows = 6
    no_cols = 3
    t_no_rows = no_rows + no_rows_aligned + 3
    fig, ax = plt.subplots(t_no_rows, no_cols, figsize=(scale * 3 * no_cols, scale * 3 * t_no_rows), gridspec_kw = {'wspace':0, 'hspace':0})
    amp = np.max(np.abs(gb))
    ax[0, 0].imshow(gb, cmap='binary', vmin=-amp, vmax=amp)

    k = 2
    for i in range(no_rows_aligned):
        for j in range(no_cols):
            amp = np.max(np.abs(GBP[gp_aligned[j]]))
            ax[k, j].imshow(GBP[gp_aligned[j]], cmap='binary', vmin=-amp, vmax=amp)
        k = k + 1

    k = k + 1
    for i in range(no_rows):
        for j in range(no_cols):
            amp = np.max(np.abs(GBP[gp_quasi[j + i * no_cols]]))
            ax[k, j].imshow(GBP[gp_quasi[j + i * no_cols]], cmap='binary', vmin=-amp, vmax=amp)
        k = k + 1
    
    for i in range(t_no_rows):
        for j in range(no_cols):
            ax[i, j].set_aspect('auto')
            ax[i, j].set_xticks([])
            ax[i, j].set_yticks([])
    ax[0, 1].set_axis_off()
    ax[0, 2].set_axis_off()
    for j in range(no_cols):
        ax[1, j].set_axis_off()
        ax[3, j].set_axis_off()
            

    if save:
        out_file = 'Gabor.pdf'
        fig.savefig(out_file, transparent=True)
    